In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/09 20:59:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/09 20:59:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


## Question 1:

In [3]:
spark.version

'3.3.2'

## Question 2:

In [4]:
# !wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-06.csv.gz

In [5]:
# !gunzip fhvhv_tripdata_2021-06.csv.gz

In [6]:
df = spark.read.options(header=True, inferSchema=True).csv("fhvhv_tripdata_2021-06.csv")

df = df.repartition(12)

df.write.parquet("data/pq/fhvhv/2021/06/")

In [7]:
!ls -lh data/pq/fhvhv/2021/06/part-00000-3226d84e-6321-4a7d-b6e8-9c1a435f73cb-c000.snappy.parquet

-rw-r--r-- 1 prajwal prajwal 23M Mar  9 21:00 data/pq/fhvhv/2021/06/part-00000-3226d84e-6321-4a7d-b6e8-9c1a435f73cb-c000.snappy.parquet


## Question 3:

In [8]:
df.withColumn("pickup_date", F.to_date(df.pickup_datetime)).filter(
    "pickup_date = '2021-06-15'"
).count()

452470

## Question 4:

In [11]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'SR_Flag',
 'Affiliated_base_number']

In [12]:
(
    df.withColumn(
        "duration", df.dropoff_datetime.cast("long") - df.pickup_datetime.cast("long")
    )
    .withColumn("pickup_date", F.to_date(df.pickup_datetime))
    .groupBy("pickup_date")
    .max("duration")
    .orderBy("max(duration)", ascending=False)
    .limit(5)
    .show()
)

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2021-06-25|       240764|
| 2021-06-22|        91979|
| 2021-06-27|        71931|
| 2021-06-26|        65510|
| 2021-06-23|        59281|
+-----------+-------------+



In [14]:
240764 / 60 / 60

66.87888888888888

## Question 5:

4040

## Question 6:

In [15]:
df_zones = spark.read.parquet("zones")

In [21]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [22]:
df.show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B02864|2021-06-04 14:18:02|2021-06-04 15:04:26|         231|         138|      N|                B02864|
|              B02884|2021-06-03 12:30:12|2021-06-03 12:41:25|         207|         170|      N|                B02884|
|              B02869|2021-06-02 11:47:07|2021-06-02 12:20:31|         231|         237|      N|                B02869|
|              B02510|2021-06-04 21:40:42|2021-06-04 22:14:57|         140|          61|      N|                  null|
|              B02864|2021-06-04 21:40:35|2021-06-04 22:02:21|         262|          33|      N|                B02864|
|              B02510|2021-06-03 14:19:5

In [19]:
df_zones.registerTempTable("zones")
df.registerTempTable("fhvhv_2021_06")

In [27]:
df.groupby("PULocationID").count().orderBy("count", ascending=False).limit(1).show()

+------------+------+
|PULocationID| count|
+------------+------+
|          61|231279|
+------------+------+



In [30]:
(
    df_zones.select(
        "LocationID",
        "Zone",
    )
    .filter(df_zones.LocationID == 61)
    .show()
)

+----------+-------------------+
|LocationID|               Zone|
+----------+-------------------+
|        61|Crown Heights North|
+----------+-------------------+

